In [1]:
!pip install tqdm


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB 1.9 MB/s eta 0:01:53
   ---------------------------------------- 0.1/216.1 MB 1.6 MB/s eta 0:02:19
   ---------------------------------------- 0.3/216.1 MB 2.7 MB/s eta 0:01:20
   ---------------------------------------- 0.7/216.1 MB 3.8 MB/s eta 0:00:57
   ---------------------------------------- 1.2/216.1 MB 4.9 MB/s eta 0:00:44
   ---------------------------------------- 1.5/216.1 MB 5.6 MB/s eta 0:00:39
   ---------------------------------------- 1.6/216.1 MB 4.9 MB/s eta 0:00:44
   ---------------------------------------- 1.8/216.1 MB 4.9 MB/s eta 0:00:45
   ---------------------------------------- 2.0/216.1 MB 5.2 MB/s eta 0:00:42
   ---------------------------------------- 2.0/216.1 MB 5.2 MB/s eta 0:00:42
   ---------------------------------------- 2.5/216.1 MB 5.1 MB/s eta 0


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install transformers torchvision seaborn opencv-python


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.5 MB 4.8 MB/s eta 0:00:03
    --------------------------------------- 0.2/10.5 MB 3.0 MB/s eta 0:00:04
   - -------------------------------------- 0.5/10.5 MB 3.9 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/10.5 MB 3.9 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/10.5 MB 3.7 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/10.5 MB 4.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/10.5 MB 5.5 MB/s eta 0:00:02
   -------- ------------------------------- 2.3/10.5 MB 6.1 MB/s eta 0:00:02
   ---------- ----------------------------- 2.8/10.5 MB 6.6 MB/s eta 0:00:02
   ----


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Part 1: Import Libraries and Setup
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from transformers import ViTImageProcessor, ViTForImageClassification
import warnings
warnings.filterwarnings('ignore')

AttributeError: module 'torch' has no attribute 'version'

In [15]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [16]:
# Part 2: Dataset Configuration and Custom Dataset Class

# Configuration - UPDATE THESE PATHS TO YOUR ACTUAL PATHS
CONFIG = {
    'schizophrenia_folder': 'D:/HHT/S',  # Update this path
    'healthy_folder': 'D:/HHT/H',              # Update this path
    'batch_size': 16,
    'num_epochs': 50,
    'learning_rate': 2e-5,
    'image_size': 224,
    'num_folds': 5,
    'num_classes': 2,
    'model_name': 'google/vit-base-patch16-224-in21k'
}

class EEGDataset(Dataset):
    def __init__(self, image_paths, labels, processor, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        # Apply transforms if provided
        if self.transform:
            image = self.transform(image)
        
        # Process with ViT processor
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)
        
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return {
            'pixel_values': pixel_values,
            'labels': label
        }

# Data augmentation transforms
train_transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
])

val_transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
])

print("Dataset class and transforms defined successfully!")

Dataset class and transforms defined successfully!


In [17]:
# Part 3: Data Loading and Preparation

def load_data_paths(schizophrenia_folder, healthy_folder):
    """Load image paths and create labels"""
    image_paths = []
    labels = []
    
    # Load schizophrenia images (label = 1)
    if os.path.exists(schizophrenia_folder):
        schiz_files = [f for f in os.listdir(schizophrenia_folder) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for file in schiz_files:
            image_paths.append(os.path.join(schizophrenia_folder, file))
            labels.append(1)
        print(f"Loaded {len(schiz_files)} schizophrenia images")
    else:
        print(f"Warning: Schizophrenia folder not found at {schizophrenia_folder}")
    
    # Load healthy images (label = 0)
    if os.path.exists(healthy_folder):
        healthy_files = [f for f in os.listdir(healthy_folder) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for file in healthy_files:
            image_paths.append(os.path.join(healthy_folder, file))
            labels.append(0)
        print(f"Loaded {len(healthy_files)} healthy images")
    else:
        print(f"Warning: Healthy folder not found at {healthy_folder}")
    
    return image_paths, labels

# Load the data
print("Loading data paths...")
image_paths, labels = load_data_paths(CONFIG['schizophrenia_folder'], CONFIG['healthy_folder'])

print(f"Total images loaded: {len(image_paths)}")
print(f"Class distribution:")
print(f"  Healthy (0): {labels.count(0)}")
print(f"  Schizophrenia (1): {labels.count(1)}")

# Convert to numpy arrays for easier handling
image_paths = np.array(image_paths)
labels = np.array(labels)

# Initialize ViT processor
processor = ViTImageProcessor.from_pretrained(CONFIG['model_name'])
print("ViT processor initialized successfully!")

Loading data paths...
Loaded 5146 schizophrenia images
Loaded 4235 healthy images
Total images loaded: 9381
Class distribution:
  Healthy (0): 4235
  Schizophrenia (1): 5146
ViT processor initialized successfully!


In [18]:
# Part 4: Custom ViT Model and Training Setup

class CustomViTModel(nn.Module):
    def __init__(self, model_name, num_classes):
        super(CustomViTModel, self).__init__()
        # Load pre-trained ViT model
        self.vit = ViTForImageClassification.from_pretrained(
            model_name,
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )
        
        # The warning about uninitialized weights is normal and expected
        # The classifier layer is randomly initialized for fine-tuning
        
    def forward(self, pixel_values, labels=None):
        outputs = self.vit(pixel_values=pixel_values, labels=labels)
        return outputs

class EEGTrainer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.model.to(device)
        
    def train_epoch(self, train_loader, optimizer, criterion):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch in tqdm(train_loader, desc="Training"):
            pixel_values = batch['pixel_values'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            optimizer.zero_grad()
            
            outputs = self.model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            # Calculate accuracy
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
        
        avg_loss = total_loss / len(train_loader)
        accuracy = correct / total
        return avg_loss, accuracy
    
    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0
        correct = 0
        total = 0
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                pixel_values = batch['pixel_values'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                outputs = self.model(pixel_values=pixel_values, labels=labels)
                loss = outputs.loss
                
                total_loss += loss.item()
                
                predictions = torch.argmax(outputs.logits, dim=-1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
                
                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(val_loader)
        accuracy = correct / total
        return avg_loss, accuracy, all_predictions, all_labels

print("Custom ViT model and trainer classes defined successfully!")

Custom ViT model and trainer classes defined successfully!


In [ ]:
# Part 5: 5-Fold Cross Validation Training Loop

def plot_confusion_matrix(y_true, y_pred, fold_num):
    """Plot confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Healthy', 'Schizophrenia'],
                yticklabels=['Healthy', 'Schizophrenia'])
    plt.title(f'Confusion Matrix - Fold {fold_num}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    return cm

def save_model(model, fold_num, accuracy, save_dir='saved_models'):
    """Save the best model"""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    model_path = os.path.join(save_dir, f'best_vit_model_fold_{fold_num}_acc_{accuracy:.4f}.pth')
    torch.save({
        'model_state_dict': model.state_dict(),
        'accuracy': accuracy,
        'fold': fold_num
    }, model_path)
    print(f"Model saved: {model_path}")
    return model_path

# Initialize cross-validation
kfold = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=42)

# Store results
fold_results = []
all_fold_predictions = []
all_fold_labels = []
best_models = []

print("Starting 5-Fold Cross Validation...")
print("="*50)

for fold, (train_idx, val_idx) in enumerate(kfold.split(image_paths, labels)):
    print(f"\nFOLD {fold + 1}/{CONFIG['num_folds']}")
    print("-" * 30)
    
    # Split data
    train_paths, val_paths = image_paths[train_idx], image_paths[val_idx]
    train_labels, val_labels = labels[train_idx], labels[val_idx]
    
    print(f"Train samples: {len(train_paths)} | Validation samples: {len(val_paths)}")
    
    # Create datasets
    train_dataset = EEGDataset(train_paths, train_labels, processor, train_transform)
    val_dataset = EEGDataset(val_paths, val_labels, processor, val_transform)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], 
                            shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], 
                          shuffle=False, num_workers=2, pin_memory=True)
    
    # Initialize model for this fold
    model = CustomViTModel(CONFIG['model_name'], CONFIG['num_classes'])
    trainer = EEGTrainer(model, device)
    
    # Setup optimizer and criterion
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['num_epochs'])
    
    # Training loop
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in range(CONFIG['num_epochs']):
        # Train
        train_loss, train_acc = trainer.train_epoch(train_loader, optimizer, None)
        
        # Validate
        val_loss, val_acc, val_preds, val_true = trainer.validate(val_loader)
        
        # Update scheduler
        scheduler.step()
        
        # Store metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = save_model(model, fold + 1, val_acc)
            best_predictions = val_preds.copy()
            best_true_labels = val_true.copy()
        
        # Print progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{CONFIG['num_epochs']} - "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # Store fold results
    fold_results.append({
        'fold': fold + 1,
        'best_val_accuracy': best_val_acc,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs,
        'model_path': best_model_path
    })
    
    all_fold_predictions.extend(best_predictions)
    all_fold_labels.extend(best_true_labels)
    
    # Plot confusion matrix for this fold
    cm = plot_confusion_matrix(best_true_labels, best_predictions, fold + 1)
    
    # Print fold summary
    print(f"\nFold {fold + 1} Results:")
    print(f"Best Validation Accuracy: {best_val_acc:.4f}")
    print(f"Classification Report:")
    print(classification_report(best_true_labels, best_predictions, 
                              target_names=['Healthy', 'Schizophrenia']))

print("\n" + "="*50)
print("Cross-Validation Training Completed!")
print("="*50)

Starting 5-Fold Cross Validation...

FOLD 1/5
------------------------------
Train samples: 7504 | Validation samples: 1877


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training:   0%|          | 0/469 [00:00<?, ?it/s]